In [1]:
!pip3 install gluoncv

In [1]:
from __future__ import division
from __future__ import print_function

from IPython.display import clear_output, Markdown, display
import json

from gluoncv import model_zoo, data, utils
import mxnet as mx
import time

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
plt.rcParams["figure.figsize"] = (20,20)

In [2]:
import os
import sys
path = os.path.abspath(os.path.join(os.path.dirname("panorama_sdk"), '../..'))
sys.path.insert(1, path + '/panorama_sdk')

import jupyter_utils

jupyter_utils.declare_globals({'mxnet_modelzoo_example': False, 
                               'custom_model': True, 'task':'object_detection', 'framework':'MXNET'})

import panoramasdk

print('Loading Model')
model = panoramasdk.model()
model.open('deploy_ssd_vgg16_reduced_512', 1)
print('Model Loaded')

Loading Model
Model Loaded


/home/ec2-user/anaconda3/envs/mxnet_latest_p37/gpu_cuda10.1/lib/python3.7/site-packages/mxnet/gluon/block.py:1577: UserWarning: Cannot decide type for the following arguments. Consider providing them as input:
	data: None
  input_sym_arg_type = in_param.infer_type()[0]


In [3]:
def preprocess(img, size):

    resized = cv2.resize(img, (size, size))
    mean = [0.485, 0.456, 0.406]  # RGB
    std = [0.229, 0.224, 0.225]  # RGB

    img = resized.astype(np.float32) / 255.  # converting array of ints to floats
    img_a = img[:, :, 0]
    img_b = img[:, :, 1]
    img_c = img[:, :, 2]

    # Extracting single channels from 3 channel image
    # The above code could also be replaced with cv2.split(img) << which will return 3 numpy arrays (using opencv)
    # normalizing per channel data:
    img_a = (img_a - mean[0]) / std[0]
    img_b = (img_b - mean[1]) / std[1]
    img_c = (img_c - mean[2]) / std[2]

    # putting the 3 channels back together:
    x1 = [[[], [], []]]
    x1[0][0] = img_a
    x1[0][1] = img_b
    x1[0][2] = img_c
    x1 = np.asarray(x1)

    return x1

In [4]:
import cv2
import numpy as np


## Panorama has a unique signature where we have to create empty arrays with the output dimensions before hand

# Create input and output arrays.
class_info = model.get_output(0)
prob_info = model.get_output(1)
rect_info = model.get_output(2)

class_array = np.empty(class_info.get_dims(), dtype=class_info.get_type())
prob_array = np.empty(prob_info.get_dims(), dtype=prob_info.get_type())
rect_array = np.empty(rect_info.get_dims(), dtype=rect_info.get_type())

filename = 'street_test.jpg'
image = cv2.imread(filename)

# Pre Process Frame
x1 = preprocess(image, 512)

# Do inference on the new frame.
model.batch(0, x1)
model.flush()

# Get the results.
resultBatchSet = model.get_result()

class_batch = resultBatchSet.get(0)
prob_batch = resultBatchSet.get(1)
rect_batch = resultBatchSet.get(2)

class_batch.get(0, class_array)
prob_batch.get(1, prob_array)
rect_batch.get(2, rect_array)

class_data = class_array[0]
prob_data = prob_array[0]
rect_data = rect_array[0]


print('Class data shape is ', class_data.shape)
print('Confidence data shape is ', prob_data.shape)
print('Bounding Boxes data shape is ',rect_data.shape)

plt.imshow(image)
plt.title(filename)
plt.show()

  0%|          | 0/3124 [00:00<?, ?KB/s]

100%|██████████| 3124/3124 [00:00<00:00, 41868.66KB/s]


ValueError: not enough values to unpack (expected 3, got 1)